In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq
import regex
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Load data
path = "/content/drive/MyDrive/1661-0.txt"
text = open(path).read().lower()

In [ ]:
#Data Preprocessing
character = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(character))
indices_char = dict((i, c) for i, c in enumerate(character))
 
print(f'unique chars: {len(character)}')

unique chars: 73


In [ ]:
#chunk 40 characters with 3 sequences
seq_len = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - seq_len, step):
    sentences.append(text[i: i + seq_len ])
    next_chars.append(text[i + seq_len])
print(f'num training examples: {len(sentences)}')

num training examples: 193950


In [ ]:
# generating our features and labels
#one hot encoding
X = np.zeros((len(sentences), seq_len, len(character)), dtype=np.bool)
y = np.zeros((len(sentences), len(character)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

sentences[124]
next_chars[100]

<ipython-input-7-2f2383d5361d>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), seq_len, len(character)), dtype=np.bool)
<ipython-input-7-2f2383d5361d>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(character)), dtype=np.bool)


'e'

In [ ]:
#one hot encoded data
X[0][0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True])

In [ ]:
X.shape

(193950, 40, 73)

In [ ]:
print(X[0][0])

[ True False False ... False False False]


In [ ]:
# Creating the model
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, len(character))))
model.add(Dense(len(character)))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_13 (LSTM)              (None, 256)               337920    
                                                                 
 dense_9 (Dense)             (None, 73)                18761     
                                                                 
 activation_9 (Activation)   (None, 73)                0         
                                                                 
Total params: 356,681
Trainable params: 356,681
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Training our model
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=25,monitor='val_loss',restore_best_weights=True)
history = model.fit(X, y,
                    validation_split=0.01, 
                    batch_size=128,
                    epochs=50, 
                    shuffle=True,callbacks=[early_stopping_cb]).history

Epoch 1/50
1501/1501 [==============================] - 14s 8ms/step - loss: 2.0685 - accuracy: 0.3945 - val_loss: 2.1906 - val_accuracy: 0.3825
Epoch 2/50
1501/1501 [==============================] - 11s 7ms/step - loss: 1.6359 - accuracy: 0.5054 - val_loss: 1.9843 - val_accuracy: 0.4376
Epoch 3/50
1501/1501 [==============================] - 11s 8ms/step - loss: 1.5018 - accuracy: 0.5417 - val_loss: 1.9150 - val_accuracy: 0.4531
Epoch 4/50
1501/1501 [==============================] - 11s 7ms/step - loss: 1.4290 - accuracy: 0.5615 - val_loss: 1.8420 - val_accuracy: 0.4851
Epoch 5/50
1501/1501 [==============================] - 11s 7ms/step - loss: 1.3772 - accuracy: 0.5737 - val_loss: 1.8588 - val_accuracy: 0.4830
Epoch 6/50
1501/1501 [==============================] - 11s 7ms/step - loss: 1.3426 - accuracy: 0.5814 - val_loss: 1.8796 - val_accuracy: 0.4758
Epoch 7/50
1501/1501 [==============================] - 11s 7ms/step - loss: 1.3184 - accuracy: 0.5876 - val_loss: 1.8619 - val_ac

In [ ]:
#Model Evaluation
loss_and_acc=model.evaluate(X,y)
print("Test Loss", loss_and_acc[0])
print("Test Accuracy", loss_and_acc[1])

6061/6061 [==============================] - 22s 4ms/step - loss: 1.2741 - accuracy: 0.6089
Test Loss 1.2740947008132935
Test Accuracy 0.6089301109313965


In [ ]:
def prepare_input(text):
    x = np.zeros((1, seq_len, len(character)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
        
    return x

In [ ]:
#functions to get next probable characters
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion
def predict_completions(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]            

In [ ]:
#Prediction
quotes = [   
    "it is nothing but correct prediction by the model and got average accuracy.",
    "They work well for natural language processing tasks such as next word predi.",
    "The sequences are then padded to ensure they have the same length.",
    "After training, the accuracy of the model on the training data can be evaluated.",
    "Model was trained with long short term memory rnn."  
    
]

In [ ]:
model.save("/content/drive/MyDrive/nextword.h5")

In [ ]:
model = load_model("/content/drive/MyDrive/nextword.h5")

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_13 (LSTM)              (None, 256)               337920    
                                                                 
 dense_9 (Dense)             (None, 73)                18761     
                                                                 
 activation_9 (Activation)   (None, 73)                0         
                                                                 
Total params: 356,681
Trainable params: 356,681
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for q in quotes:
    seq = q[:40].lower()
    print(seq)
    print(predict_completions(seq, 5))
    print()

it is nothing but correct prediction by 
['the ', 'a ', 'some ', 'my ', 'his ']

they work well for natural language proc
['ised ', 'essible ', 'less ', 'onted ', 'ate ']

the sequences are then padded to ensure 
['the ', 'and ', 'which ', 'of ', 'in ']

after training, the accuracy of the mode
[' that ', 'n ', 'd ', '. ', 'r ']

model was trained with long short term m
['an ', 'e ', 'ore ', 'inutes ', 'y ']

